In [1]:
import re
import time
import os

import pandas as pd
import pylab as plt
import numpy as np
import seaborn as sns

import requests
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor

import Bio
from Bio import Phylo, SeqIO, Entrez
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq

import networkx as nx
import igraph
from ete3 import NCBITaxa

In [2]:
Entrez.email = 'burankova.y@gmail.com'

Using truncated TOL (https://itol.embl.de/) — leave only Opisthokonta.

# Truncate TOL

In [177]:
tree = Phylo.read('../data/TOL/normal_newick.txt', "newick")
Phylo.draw_ascii(tree)

                     __________ Nanoarchaeum_equitans
                   _|
                  | |  _______ Pyrobaculum_aerophilum
                  | |_|
                  |   | _____ Aeropyrum_pernix
                  |   ||
                  |    |     __ Sulfolobus_solfataricus
                  |    |____|
              ____|         |_ Sulfolobus_tokodaii
             |    |
             |    |          , Thermoplasma_volcanium
             |    |  ________|
             |    | |        | Thermoplasma_acidophilum
             |    | |
             |    | |  ____ Archaeoglobus_fulgidus
             |    | |,|
             |    |_|||  _______ Halobacterium_sp._NRC-1
             |      |||_|
             |      ||  |   , Methanosarcina_acetivorans
             |      ||  |___|
             |      ||      | Methanosarcina_mazei
             |      ||
             |       |    , Pyrococcus_furiosus
  ___________|       | ___|
 |           |       ||   |, Pyrococcus_horikoshii
 |      

In [61]:
tree.get_nonterminals()

[Clade(),
 Clade(branch_length=0.88776),
 Clade(branch_length=0.43325, comment='100', name='Archaea'),
 Clade(branch_length=0.15444, comment='100', name='Archaea'),
 Clade(branch_length=0.12018, comment='100', name='Thermoprotei'),
 Clade(branch_length=0.09462, comment='94', name='Thermoprotei'),
 Clade(branch_length=0.3372, comment='100', name='Sulfolobus'),
 Clade(branch_length=0.11833, comment='99', name='Euryarchaeota'),
 Clade(branch_length=0.66151, comment='100', name='Thermoplasma'),
 Clade(branch_length=0.06815, comment='62', name='Euryarchaeota'),
 Clade(branch_length=0.10395, comment='100', name='Euryarchaeota'),
 Clade(branch_length=0.12801, comment='100', name='Euryarchaeota'),
 Clade(branch_length=0.30588, comment='100', name='Methanosarcina'),
 Clade(branch_length=0.04469, comment='51', name='Euryarchaeota'),
 Clade(branch_length=0.3622, comment='100', name='Pyrococcus'),
 Clade(branch_length=0.02239, comment='100', name='Pyrococcus'),
 Clade(branch_length=0.06284, commen

In [73]:
small_tree = tree.common_ancestor('Eukaryota', 'Archaea')
Phylo.draw_ascii(small_tree)

                         ____________ Nanoarchaeum_equitans
                       _|
                      | |  ________ Pyrobaculum_aerophilum
                      | |_|
                      |   |  ______ Aeropyrum_pernix
                      |   |_|
                      |     |     __ Sulfolobus_solfataricus
                      |     |____|
                ______|          |__ Sulfolobus_tokodaii
               |      |
               |      |            _ Thermoplasma_volcanium
               |      |  _________|
               |      | |         |_ Thermoplasma_acidophilum
               |      | |
               |      | |  _____ Archaeoglobus_fulgidus
               |      | |,|
               |      |_|||   ________ Halobacterium_sp._NRC-1
               |        |||__|
               |        ||   |   , Methanosarcina_acetivorans
               |        ||   |___|
               |        ||       | Methanosarcina_mazei
               |        ||
               |         

In [76]:
small_tree = tree.common_ancestor('Eukaryota')
Phylo.draw_ascii(small_tree)

                        ____________________ Giardia_lamblia
                       |
                       |   _________________ Leishmania_major
_______________________|  |
                       |  |    ____________ Thalassiosira_pseudonana
                       |  |  ,|
                       |__|  ||      __________ Plasmodium_falciparum
                          |  ||_____|
                          |  |      |________ Cryptosporidium_hominis
                          |  |
                          |__|    ______________ Cyanidioschyzon_merolae
                             | __|
                             ||  |      __ Oryza_sativa
                             ||  |_____|
                             ||        |___ Arabidopsis_thaliana
                             ||
                              | ____________ Dictyostelium_discoideum
                              ||
                              ||       _________ Schizosaccharomyces_pombe
                              ||  

In [77]:
# save small tree
Phylo.write(small_tree, "../data/TOL/Eukaryota_tree.nwk", "newick")

1

In [78]:
# save list with terminal nodes of truncated tree
nodes_list = []

with open("../data/TOL/Eukaryota_tree_nodes.txt", 'w') as f:    
    for clade in small_tree.get_terminals():
        clade_str = ' '.join(clade.name.split('_'))
        nodes_list.append(clade_str)
        f.write(clade_str)
        f.write('\n')

# Download S. cerevisiae genes 

In [286]:
yeast_data = pd.read_csv('../data/yeast.txt', sep='\t')

In [288]:
genes = set(yeast_data.Tf)
genes.update(yeast_data.Tg)

with open("../data/genes_lists/yeast/S_cerevisiae_genes.txt", 'w') as f: 
    f.write("\n".join(list(genes)))

In [289]:
genes_dict = dict.fromkeys(genes)
yeasts_genes = len(genes)

In [290]:
posted_genes = []

In [213]:
YG_URL = "https://www.yeastgenome.org/search"

def fetch(gene_name):
    if genes_dict[gene_name]:
        return None, None
    
    posted_genes.append(gene_name)
    
    params = {
            'q': gene_name,
            'is_quick': 'true',
        }
    
    resp = requests.post(YG_URL, params=params)
    time.sleep(3)
    return resp.content, gene_name
    
    
pool = ThreadPoolExecutor(max_workers=10)
i = 1

for page, gene_name in pool.map(fetch, genes_dict):
    try:
        soup = BeautifulSoup(page, 'lxml')
        locusID = re.search(r'(?<=locusId: ").*(?=",\n)', soup.find('script').text).group(0)
        genes_dict[gene_name] = locusID
        
    except (IndexError, TypeError, AttributeError):
        continue
        
    finally:
        print(f'{i} / {yeasts_genes} [{i/yeasts_genes}]')
        i += 1

0 / 4441 [0.0]
1 / 4441 [0.00022517451024544022]
2 / 4441 [0.00045034902049088043]
3 / 4441 [0.0006755235307363207]
4 / 4441 [0.0009006980409817609]
5 / 4441 [0.001125872551227201]
6 / 4441 [0.0013510470614726414]
7 / 4441 [0.0015762215717180816]
8 / 4441 [0.0018013960819635217]
9 / 4441 [0.002026570592208962]
10 / 4441 [0.002251745102454402]
11 / 4441 [0.0024769196126998424]
12 / 4441 [0.002702094122945283]
13 / 4441 [0.0029272686331907227]
14 / 4441 [0.003152443143436163]
15 / 4441 [0.003377617653681603]
16 / 4441 [0.0036027921639270434]
17 / 4441 [0.003827966674172484]
18 / 4441 [0.004053141184417924]
19 / 4441 [0.004278315694663364]
20 / 4441 [0.004503490204908804]
21 / 4441 [0.0047286647151542445]
22 / 4441 [0.004953839225399685]
23 / 4441 [0.005179013735645125]
24 / 4441 [0.005404188245890566]
25 / 4441 [0.005629362756136005]
26 / 4441 [0.0058545372663814455]
27 / 4441 [0.006079711776626886]
28 / 4441 [0.006304886286872326]
29 / 4441 [0.006530060797117767]
30 / 4441 [0.0067552353

356 / 4441 [0.08016212564737672]
357 / 4441 [0.08038730015762216]
358 / 4441 [0.0806124746678676]
359 / 4441 [0.08083764917811304]
360 / 4441 [0.08106282368835847]
361 / 4441 [0.08128799819860391]
362 / 4441 [0.08151317270884936]
363 / 4441 [0.0817383472190948]
364 / 4441 [0.08196352172934024]
365 / 4441 [0.08218869623958568]
366 / 4441 [0.08241387074983111]
367 / 4441 [0.08263904526007657]
368 / 4441 [0.082864219770322]
369 / 4441 [0.08308939428056744]
370 / 4441 [0.08331456879081288]
371 / 4441 [0.08353974330105832]
372 / 4441 [0.08376491781130375]
373 / 4441 [0.0839900923215492]
374 / 4441 [0.08421526683179464]
375 / 4441 [0.08444044134204008]
376 / 4441 [0.08466561585228552]
377 / 4441 [0.08489079036253096]
378 / 4441 [0.0851159648727764]
379 / 4441 [0.08534113938302185]
380 / 4441 [0.08556631389326728]
381 / 4441 [0.08579148840351272]
382 / 4441 [0.08601666291375816]
383 / 4441 [0.0862418374240036]
384 / 4441 [0.08646701193424905]
385 / 4441 [0.08669218644449449]
386 / 4441 [0.086

606 / 4441 [0.13645575320873676]
607 / 4441 [0.13668092771898221]
608 / 4441 [0.13690610222922764]
609 / 4441 [0.1371312767394731]
610 / 4441 [0.13735645124971854]
611 / 4441 [0.13758162575996397]
612 / 4441 [0.13780680027020942]
613 / 4441 [0.13803197478045484]
614 / 4441 [0.1382571492907003]
615 / 4441 [0.13848232380094574]
616 / 4441 [0.13870749831119117]
617 / 4441 [0.13893267282143662]
618 / 4441 [0.13915784733168204]
619 / 4441 [0.1393830218419275]
620 / 4441 [0.13960819635217295]
621 / 4441 [0.13983337086241837]
622 / 4441 [0.14005854537266382]
623 / 4441 [0.14028371988290925]
624 / 4441 [0.1405088943931547]
625 / 4441 [0.14073406890340012]
626 / 4441 [0.14095924341364557]
627 / 4441 [0.14118441792389103]
628 / 4441 [0.14140959243413645]
629 / 4441 [0.1416347669443819]
630 / 4441 [0.14185994145462733]
631 / 4441 [0.14208511596487278]
632 / 4441 [0.14231029047511823]
633 / 4441 [0.14253546498536365]
634 / 4441 [0.1427606394956091]
635 / 4441 [0.14298581400585453]
636 / 4441 [0.14

905 / 4441 [0.2037829317721234]
906 / 4441 [0.20400810628236885]
907 / 4441 [0.20423328079261427]
908 / 4441 [0.20445845530285972]
909 / 4441 [0.20468362981310514]
910 / 4441 [0.2049088043233506]
911 / 4441 [0.20513397883359605]
912 / 4441 [0.20535915334384147]
913 / 4441 [0.20558432785408692]
914 / 4441 [0.20580950236433235]
915 / 4441 [0.2060346768745778]
916 / 4441 [0.20625985138482325]
917 / 4441 [0.20648502589506867]
918 / 4441 [0.20671020040531413]
919 / 4441 [0.20693537491555955]
920 / 4441 [0.207160549425805]
921 / 4441 [0.20738572393605043]
922 / 4441 [0.20761089844629588]
923 / 4441 [0.20783607295654133]
924 / 4441 [0.20806124746678675]
925 / 4441 [0.2082864219770322]
926 / 4441 [0.20851159648727763]
927 / 4441 [0.20873677099752308]
928 / 4441 [0.20896194550776853]
929 / 4441 [0.20918712001801396]
930 / 4441 [0.2094122945282594]
931 / 4441 [0.20963746903850483]
932 / 4441 [0.20986264354875028]
933 / 4441 [0.21008781805899573]
934 / 4441 [0.21031299256924116]
935 / 4441 [0.210

1152 / 4441 [0.25940103580274715]
1153 / 4441 [0.25962621031299254]
1154 / 4441 [0.259851384823238]
1155 / 4441 [0.26007655933348345]
1156 / 4441 [0.2603017338437289]
1157 / 4441 [0.26052690835397435]
1158 / 4441 [0.26075208286421975]
1159 / 4441 [0.2609772573744652]
1160 / 4441 [0.26120243188471065]
1161 / 4441 [0.2614276063949561]
1162 / 4441 [0.26165278090520155]
1163 / 4441 [0.26187795541544695]
1164 / 4441 [0.2621031299256924]
1165 / 4441 [0.26232830443593785]
1166 / 4441 [0.2625534789461833]
1167 / 4441 [0.26277865345642876]
1168 / 4441 [0.26300382796667415]
1169 / 4441 [0.2632290024769196]
1170 / 4441 [0.26345417698716506]
1171 / 4441 [0.2636793514974105]
1172 / 4441 [0.26390452600765596]
1173 / 4441 [0.26412970051790136]
1174 / 4441 [0.2643548750281468]
1175 / 4441 [0.26458004953839226]
1176 / 4441 [0.2648052240486377]
1177 / 4441 [0.2650303985588831]
1178 / 4441 [0.26525557306912856]
1179 / 4441 [0.265480747579374]
1180 / 4441 [0.26570592208961946]
1181 / 4441 [0.2659310965998

...

3360 / 4441 [0.7565863544246791]
3361 / 4441 [0.7568115289349245]
3362 / 4441 [0.75703670344517]
3363 / 4441 [0.7572618779554154]
3364 / 4441 [0.7574870524656608]
3365 / 4441 [0.7577122269759063]
3366 / 4441 [0.7579374014861517]
3367 / 4441 [0.7581625759963972]
3368 / 4441 [0.7583877505066426]
3369 / 4441 [0.758612925016888]
3370 / 4441 [0.7588380995271335]
3371 / 4441 [0.7590632740373789]
3372 / 4441 [0.7592884485476245]
3373 / 4441 [0.7595136230578698]
3374 / 4441 [0.7597387975681152]
3375 / 4441 [0.7599639720783607]
3376 / 4441 [0.7601891465886061]
3377 / 4441 [0.7604143210988517]
3378 / 4441 [0.760639495609097]
3379 / 4441 [0.7608646701193424]
3380 / 4441 [0.761089844629588]
3381 / 4441 [0.7613150191398333]
3382 / 4441 [0.7615401936500789]
3383 / 4441 [0.7617653681603243]
3384 / 4441 [0.7619905426705696]
3385 / 4441 [0.7622157171808152]
3386 / 4441 [0.7624408916910606]
3387 / 4441 [0.7626660662013061]
3388 / 4441 [0.7628912407115515]
3389 / 4441 [0.7631164152217969]
3390 / 4441 [0.

3610 / 4441 [0.8128799819860392]
3611 / 4441 [0.8131051564962847]
3612 / 4441 [0.8133303310065301]
3613 / 4441 [0.8135555055167755]
3614 / 4441 [0.813780680027021]
3615 / 4441 [0.8140058545372664]
3616 / 4441 [0.8142310290475118]
3617 / 4441 [0.8144562035577573]
3618 / 4441 [0.8146813780680027]
3619 / 4441 [0.8149065525782482]
3620 / 4441 [0.8151317270884936]
3621 / 4441 [0.815356901598739]
3622 / 4441 [0.8155820761089845]
3623 / 4441 [0.8158072506192299]
3624 / 4441 [0.8160324251294754]
3625 / 4441 [0.8162575996397208]
3626 / 4441 [0.8164827741499662]
3627 / 4441 [0.8167079486602117]
3628 / 4441 [0.8169331231704571]
3629 / 4441 [0.8171582976807026]
3630 / 4441 [0.817383472190948]
3631 / 4441 [0.8176086467011934]
3632 / 4441 [0.8178338212114389]
3633 / 4441 [0.8180589957216843]
3634 / 4441 [0.8182841702319298]
3635 / 4441 [0.8185093447421752]
3636 / 4441 [0.8187345192524206]
3637 / 4441 [0.8189596937626661]
3638 / 4441 [0.8191848682729115]
3639 / 4441 [0.819410042783157]
3640 / 4441 [0

3861 / 4441 [0.8693987840576447]
3862 / 4441 [0.8696239585678901]
3863 / 4441 [0.8698491330781356]
3864 / 4441 [0.870074307588381]
3865 / 4441 [0.8702994820986264]
3866 / 4441 [0.8705246566088719]
3867 / 4441 [0.8707498311191173]
3868 / 4441 [0.8709750056293628]
3869 / 4441 [0.8712001801396082]
3870 / 4441 [0.8714253546498536]
3871 / 4441 [0.8716505291600991]
3872 / 4441 [0.8718757036703445]
3873 / 4441 [0.87210087818059]
3874 / 4441 [0.8723260526908354]
3875 / 4441 [0.8725512272010808]
3876 / 4441 [0.8727764017113263]
3877 / 4441 [0.8730015762215717]
3878 / 4441 [0.8732267507318171]
3879 / 4441 [0.8734519252420626]
3880 / 4441 [0.873677099752308]
3881 / 4441 [0.8739022742625535]
3882 / 4441 [0.8741274487727989]
3883 / 4441 [0.8743526232830443]
3884 / 4441 [0.8745777977932898]
3885 / 4441 [0.8748029723035352]
3886 / 4441 [0.8750281468137807]
3887 / 4441 [0.8752533213240261]
3888 / 4441 [0.8754784958342715]
3889 / 4441 [0.875703670344517]
3890 / 4441 [0.8759288448547624]
3891 / 4441 [0.

4111 / 4441 [0.9256924116190047]
4112 / 4441 [0.9259175861292501]
4113 / 4441 [0.9261427606394956]
4114 / 4441 [0.926367935149741]
4115 / 4441 [0.9265931096599865]
4116 / 4441 [0.9268182841702319]
4117 / 4441 [0.9270434586804773]
4118 / 4441 [0.9272686331907228]
4119 / 4441 [0.9274938077009682]
4120 / 4441 [0.9277189822112137]
4121 / 4441 [0.9279441567214591]
4122 / 4441 [0.9281693312317045]
4123 / 4441 [0.92839450574195]
4124 / 4441 [0.9286196802521954]
4125 / 4441 [0.9288448547624409]
4126 / 4441 [0.9290700292726863]
4127 / 4441 [0.9292952037829317]
4128 / 4441 [0.9295203782931772]
4129 / 4441 [0.9297455528034226]
4130 / 4441 [0.9299707273136681]
4131 / 4441 [0.9301959018239135]
4132 / 4441 [0.9304210763341589]
4133 / 4441 [0.9306462508444044]
4134 / 4441 [0.9308714253546498]
4135 / 4441 [0.9310965998648953]
4136 / 4441 [0.9313217743751407]
4137 / 4441 [0.9315469488853861]
4138 / 4441 [0.9317721233956316]
4139 / 4441 [0.931997297905877]
4140 / 4441 [0.9322224724161225]
4141 / 4441 [0

4363 / 4441 [0.9824363882008557]
4364 / 4441 [0.982661562711101]
4365 / 4441 [0.9828867372213466]
4366 / 4441 [0.983111911731592]
4367 / 4441 [0.9833370862418375]
4368 / 4441 [0.9835622607520829]
4369 / 4441 [0.9837874352623283]
4370 / 4441 [0.9840126097725738]
4371 / 4441 [0.9842377842828192]
4372 / 4441 [0.9844629587930647]
4373 / 4441 [0.9846881333033101]
4374 / 4441 [0.9849133078135555]
4375 / 4441 [0.985138482323801]
4376 / 4441 [0.9853636568340464]
4377 / 4441 [0.9855888313442919]
4378 / 4441 [0.9858140058545373]
4379 / 4441 [0.9860391803647827]
4380 / 4441 [0.9862643548750282]
4381 / 4441 [0.9864895293852736]
4382 / 4441 [0.9867147038955191]
4383 / 4441 [0.9869398784057645]
4384 / 4441 [0.9871650529160099]
4385 / 4441 [0.9873902274262554]
4386 / 4441 [0.9876154019365008]
4387 / 4441 [0.9878405764467463]
4388 / 4441 [0.9880657509569917]
4389 / 4441 [0.9882909254672371]
4390 / 4441 [0.9885160999774826]
4391 / 4441 [0.988741274487728]
4392 / 4441 [0.9889664489979735]
4393 / 4441 [0

In [216]:
i = 0
for gene in genes_dict:
    if not genes_dict[gene]:
        i += 1
i

211

In [247]:
search_genes = []
downloaded_genes = []

In [250]:
def fetch(gene_name):
    if gene_name in downloaded_genes:
        return None, None
    
    if not genes_dict[gene_name]:
        search_genes.append(gene_name)
        return None, None
    
    URL = os.path.join('https://www.yeastgenome.org/backend/locus/', genes_dict[gene_name], 'sequence_details')
    
    new_resp = requests.get(URL)
    search_genes.append(gene_name)
    time.sleep(3)
    
    return new_resp.content, gene_name
    
    
pool = ThreadPoolExecutor(max_workers=10)
i = 1

for page, gene_name in pool.map(fetch, genes_dict):
    try:
        soup = BeautifulSoup(page, 'lxml')
        
        dnas = soup.find('p').text.split('coding_dna')[1]
        dna = dnas.split('"residues": "')[1].split('",')[0]
        
        with open("../data/genes_lists/S_cerevisiae_genes_seq.fa", "a") as output_handle:
            SeqIO.write(SeqRecord(Seq(dna), id = gene_name, description='S.cerevisiae'), output_handle, "fasta")
        
        downloaded_genes.append(gene_name)
        
    except (IndexError, TypeError, AttributeError):
        continue
        
    finally:
        print(f'{i} / {yeasts_genes} [{i/yeasts_genes}]')
        i += 1

0 / 4441 [0.0]
1 / 4441 [0.00022517451024544022]
2 / 4441 [0.00045034902049088043]
3 / 4441 [0.0006755235307363207]
4 / 4441 [0.0009006980409817609]
5 / 4441 [0.001125872551227201]
6 / 4441 [0.0013510470614726414]
7 / 4441 [0.0015762215717180816]
8 / 4441 [0.0018013960819635217]
9 / 4441 [0.002026570592208962]
10 / 4441 [0.002251745102454402]
11 / 4441 [0.0024769196126998424]
12 / 4441 [0.002702094122945283]
13 / 4441 [0.0029272686331907227]
14 / 4441 [0.003152443143436163]
15 / 4441 [0.003377617653681603]
16 / 4441 [0.0036027921639270434]
17 / 4441 [0.003827966674172484]
18 / 4441 [0.004053141184417924]
19 / 4441 [0.004278315694663364]
20 / 4441 [0.004503490204908804]
21 / 4441 [0.0047286647151542445]
22 / 4441 [0.004953839225399685]
23 / 4441 [0.005179013735645125]
24 / 4441 [0.005404188245890566]
25 / 4441 [0.005629362756136005]
26 / 4441 [0.0058545372663814455]
27 / 4441 [0.006079711776626886]
28 / 4441 [0.006304886286872326]
29 / 4441 [0.006530060797117767]
30 / 4441 [0.0067552353

250 / 4441 [0.056293627561360055]
251 / 4441 [0.05651880207160549]
252 / 4441 [0.05674397658185094]
253 / 4441 [0.056969151092096375]
254 / 4441 [0.05719432560234181]
255 / 4441 [0.05741950011258726]
256 / 4441 [0.057644674622832695]
257 / 4441 [0.05786984913307813]
258 / 4441 [0.05809502364332358]
259 / 4441 [0.058320198153569015]
260 / 4441 [0.05854537266381445]
261 / 4441 [0.0587705471740599]
262 / 4441 [0.058995721684305336]
263 / 4441 [0.05922089619455078]
264 / 4441 [0.05944607070479622]
265 / 4441 [0.059671245215041656]
266 / 4441 [0.0598964197252871]
267 / 4441 [0.06012159423553254]
268 / 4441 [0.060346768745777976]
269 / 4441 [0.06057194325602342]
270 / 4441 [0.06079711776626886]
271 / 4441 [0.061022292276514296]
272 / 4441 [0.06124746678675974]
273 / 4441 [0.06147264129700518]
274 / 4441 [0.06169781580725062]
275 / 4441 [0.06192299031749606]
276 / 4441 [0.0621481648277415]
277 / 4441 [0.06237333933798694]
278 / 4441 [0.06259851384823238]
279 / 4441 [0.06282368835847782]
280 /

500 / 4441 [0.11258725512272011]
501 / 4441 [0.11281242963296555]
502 / 4441 [0.11303760414321098]
503 / 4441 [0.11326277865345642]
504 / 4441 [0.11348795316370187]
505 / 4441 [0.11371312767394731]
506 / 4441 [0.11393830218419275]
507 / 4441 [0.11416347669443819]
508 / 4441 [0.11438865120468363]
509 / 4441 [0.11461382571492908]
510 / 4441 [0.11483900022517451]
511 / 4441 [0.11506417473541995]
512 / 4441 [0.11528934924566539]
513 / 4441 [0.11551452375591083]
514 / 4441 [0.11573969826615627]
515 / 4441 [0.11596487277640172]
516 / 4441 [0.11619004728664716]
517 / 4441 [0.11641522179689259]
518 / 4441 [0.11664039630713803]
519 / 4441 [0.11686557081738347]
520 / 4441 [0.1170907453276289]
521 / 4441 [0.11731591983787436]
522 / 4441 [0.1175410943481198]
523 / 4441 [0.11776626885836523]
524 / 4441 [0.11799144336861067]
525 / 4441 [0.11821661787885611]
526 / 4441 [0.11844179238910156]
527 / 4441 [0.118666966899347]
528 / 4441 [0.11889214140959244]
529 / 4441 [0.11911731591983787]
530 / 4441 [0.

751 / 4441 [0.16910605719432562]
752 / 4441 [0.16933123170457104]
753 / 4441 [0.1695564062148165]
754 / 4441 [0.16978158072506191]
755 / 4441 [0.17000675523530737]
756 / 4441 [0.1702319297455528]
757 / 4441 [0.17045710425579824]
758 / 4441 [0.1706822787660437]
759 / 4441 [0.17090745327628912]
760 / 4441 [0.17113262778653457]
761 / 4441 [0.17135780229678]
762 / 4441 [0.17158297680702544]
763 / 4441 [0.1718081513172709]
764 / 4441 [0.17203332582751632]
765 / 4441 [0.17225850033776177]
766 / 4441 [0.1724836748480072]
767 / 4441 [0.17270884935825265]
768 / 4441 [0.1729340238684981]
769 / 4441 [0.17315919837874352]
770 / 4441 [0.17338437288898897]
771 / 4441 [0.1736095473992344]
772 / 4441 [0.17383472190947985]
773 / 4441 [0.17405989641972527]
774 / 4441 [0.17428507092997073]
775 / 4441 [0.17451024544021618]
776 / 4441 [0.1747354199504616]
777 / 4441 [0.17496059446070705]
778 / 4441 [0.17518576897095248]
779 / 4441 [0.17541094348119793]
780 / 4441 [0.17563611799144338]
781 / 4441 [0.1758612

1003 / 4441 [0.22585003377617655]
1004 / 4441 [0.22607520828642197]
1005 / 4441 [0.22630038279666742]
1006 / 4441 [0.22652555730691284]
1007 / 4441 [0.2267507318171583]
1008 / 4441 [0.22697590632740375]
1009 / 4441 [0.22720108083764917]
1010 / 4441 [0.22742625534789462]
1011 / 4441 [0.22765142985814005]
1012 / 4441 [0.2278766043683855]
1013 / 4441 [0.22810177887863095]
1014 / 4441 [0.22832695338887637]
1015 / 4441 [0.22855212789912183]
1016 / 4441 [0.22877730240936725]
1017 / 4441 [0.2290024769196127]
1018 / 4441 [0.22922765142985815]
1019 / 4441 [0.22945282594010358]
1020 / 4441 [0.22967800045034903]
1021 / 4441 [0.22990317496059445]
1022 / 4441 [0.2301283494708399]
1023 / 4441 [0.23035352398108533]
1024 / 4441 [0.23057869849133078]
1025 / 4441 [0.23080387300157623]
1026 / 4441 [0.23102904751182166]
1027 / 4441 [0.2312542220220671]
1028 / 4441 [0.23147939653231253]
1029 / 4441 [0.23170457104255798]
1030 / 4441 [0.23192974555280343]
1031 / 4441 [0.23215492006304886]
1032 / 4441 [0.2323

...

3240 / 4441 [0.7295654131952263]
3241 / 4441 [0.7297905877054718]
3242 / 4441 [0.7300157622157172]
3243 / 4441 [0.7302409367259626]
3244 / 4441 [0.7304661112362081]
3245 / 4441 [0.7306912857464535]
3246 / 4441 [0.730916460256699]
3247 / 4441 [0.7311416347669444]
3248 / 4441 [0.7313668092771898]
3249 / 4441 [0.7315919837874353]
3250 / 4441 [0.7318171582976807]
3251 / 4441 [0.7320423328079262]
3252 / 4441 [0.7322675073181716]
3253 / 4441 [0.732492681828417]
3254 / 4441 [0.7327178563386625]
3255 / 4441 [0.7329430308489079]
3256 / 4441 [0.7331682053591534]
3257 / 4441 [0.7333933798693988]
3258 / 4441 [0.7336185543796442]
3259 / 4441 [0.7338437288898897]
3260 / 4441 [0.7340689034001351]
3261 / 4441 [0.7342940779103806]
3262 / 4441 [0.734519252420626]
3263 / 4441 [0.7347444269308714]
3264 / 4441 [0.7349696014411169]
3265 / 4441 [0.7351947759513623]
3266 / 4441 [0.7354199504616078]
3267 / 4441 [0.7356451249718532]
3268 / 4441 [0.7358702994820986]
3269 / 4441 [0.7360954739923441]
3270 / 4441 [

3490 / 4441 [0.7858590407565863]
3491 / 4441 [0.7860842152668318]
3492 / 4441 [0.7863093897770772]
3493 / 4441 [0.7865345642873227]
3494 / 4441 [0.7867597387975681]
3495 / 4441 [0.7869849133078135]
3496 / 4441 [0.787210087818059]
3497 / 4441 [0.7874352623283044]
3498 / 4441 [0.7876604368385499]
3499 / 4441 [0.7878856113487953]
3500 / 4441 [0.7881107858590407]
3501 / 4441 [0.7883359603692862]
3502 / 4441 [0.7885611348795316]
3503 / 4441 [0.7887863093897771]
3504 / 4441 [0.7890114839000225]
3505 / 4441 [0.7892366584102679]
3506 / 4441 [0.7894618329205134]
3507 / 4441 [0.7896870074307588]
3508 / 4441 [0.7899121819410043]
3509 / 4441 [0.7901373564512497]
3510 / 4441 [0.7903625309614951]
3511 / 4441 [0.7905877054717406]
3512 / 4441 [0.790812879981986]
3513 / 4441 [0.7910380544922315]
3514 / 4441 [0.7912632290024769]
3515 / 4441 [0.7914884035127223]
3516 / 4441 [0.7917135780229678]
3517 / 4441 [0.7919387525332132]
3518 / 4441 [0.7921639270434587]
3519 / 4441 [0.7923891015537041]
3520 / 4441 

3740 / 4441 [0.8421526683179464]
3741 / 4441 [0.8423778428281918]
3742 / 4441 [0.8426030173384373]
3743 / 4441 [0.8428281918486827]
3744 / 4441 [0.8430533663589281]
3745 / 4441 [0.8432785408691736]
3746 / 4441 [0.843503715379419]
3747 / 4441 [0.8437288898896644]
3748 / 4441 [0.8439540643999099]
3749 / 4441 [0.8441792389101553]
3750 / 4441 [0.8444044134204008]
3751 / 4441 [0.8446295879306462]
3752 / 4441 [0.8448547624408916]
3753 / 4441 [0.8450799369511371]
3754 / 4441 [0.8453051114613825]
3755 / 4441 [0.845530285971628]
3756 / 4441 [0.8457554604818734]
3757 / 4441 [0.8459806349921188]
3758 / 4441 [0.8462058095023643]
3759 / 4441 [0.8464309840126097]
3760 / 4441 [0.8466561585228553]
3761 / 4441 [0.8468813330331006]
3762 / 4441 [0.847106507543346]
3763 / 4441 [0.8473316820535916]
3764 / 4441 [0.847556856563837]
3765 / 4441 [0.8477820310740825]
3766 / 4441 [0.8480072055843279]
3767 / 4441 [0.8482323800945732]
3768 / 4441 [0.8484575546048188]
3769 / 4441 [0.8486827291150641]
3770 / 4441 [0

3992 / 4441 [0.8988966448997974]
3993 / 4441 [0.8991218194100428]
3994 / 4441 [0.8993469939202883]
3995 / 4441 [0.8995721684305337]
3996 / 4441 [0.8997973429407791]
3997 / 4441 [0.9000225174510246]
3998 / 4441 [0.90024769196127]
3999 / 4441 [0.9004728664715155]
4000 / 4441 [0.9006980409817609]
4001 / 4441 [0.9009232154920063]
4002 / 4441 [0.9011483900022518]
4003 / 4441 [0.9013735645124972]
4004 / 4441 [0.9015987390227427]
4005 / 4441 [0.9018239135329881]
4006 / 4441 [0.9020490880432335]
4007 / 4441 [0.902274262553479]
4008 / 4441 [0.9024994370637244]
4009 / 4441 [0.9027246115739699]
4010 / 4441 [0.9029497860842153]
4011 / 4441 [0.9031749605944607]
4012 / 4441 [0.9034001351047062]
4013 / 4441 [0.9036253096149516]
4014 / 4441 [0.903850484125197]
4015 / 4441 [0.9040756586354425]
4016 / 4441 [0.9043008331456879]
4017 / 4441 [0.9045260076559334]
4018 / 4441 [0.9047511821661788]
4019 / 4441 [0.9049763566764242]
4020 / 4441 [0.9052015311866697]
4021 / 4441 [0.9054267056969151]
4022 / 4441 [0

4242 / 4441 [0.9551902724611574]
4243 / 4441 [0.9554154469714028]
4244 / 4441 [0.9556406214816483]
4245 / 4441 [0.9558657959918937]
4246 / 4441 [0.9560909705021392]
4247 / 4441 [0.9563161450123846]
4248 / 4441 [0.95654131952263]
4249 / 4441 [0.9567664940328755]
4250 / 4441 [0.9569916685431209]
4251 / 4441 [0.9572168430533664]
4252 / 4441 [0.9574420175636118]
4253 / 4441 [0.9576671920738572]
4254 / 4441 [0.9578923665841027]
4255 / 4441 [0.9581175410943481]
4256 / 4441 [0.9583427156045936]
4257 / 4441 [0.958567890114839]
4258 / 4441 [0.9587930646250844]
4259 / 4441 [0.9590182391353299]
4260 / 4441 [0.9592434136455753]
4261 / 4441 [0.9594685881558208]
4262 / 4441 [0.9596937626660662]
4263 / 4441 [0.9599189371763116]
4264 / 4441 [0.9601441116865571]
4265 / 4441 [0.9603692861968025]
4266 / 4441 [0.960594460707048]
4267 / 4441 [0.9608196352172934]
4268 / 4441 [0.9610448097275388]
4269 / 4441 [0.9612699842377843]
4270 / 4441 [0.9614951587480297]
4271 / 4441 [0.9617203332582752]
4272 / 4441 [0

# Download blast seq IDs

fore truncated blast DB

In [145]:
with open('../data/TOL/Eukaryota_tree_nodes.txt', 'r') as file:
    nodes = file.readlines()
    for i, node in enumerate(nodes):
        nodes_list[i] = node.strip()

Create dict with taxids:

In [146]:
ncbi = NCBITaxa()
name2taxid = ncbi.get_name_translator(nodes_list)

In [147]:
print(name2taxid)

{'Anopheles gambiae': [7165], 'Arabidopsis thaliana': [3702], 'Caenorhabditis briggsae': [6238], 'Caenorhabditis elegans': [6239], 'Cryptosporidium hominis': [237895], 'Cyanidioschyzon merolae': [45157], 'Danio rerio': [7955], 'Dictyostelium discoideum': [44689], 'Drosophila melanogaster': [7227], 'Eremothecium gossypii': [33169], 'Gallus gallus': [9031], 'Homo sapiens': [9606], 'Leishmania major': [5664], 'Mus musculus': [10090], 'Oryza sativa': [4530], 'Pan troglodytes': [9598], 'Plasmodium falciparum': [5833], 'Rattus norvegicus': [10116], 'Saccharomyces cerevisiae': [4932], 'Schizosaccharomyces pombe': [4896], 'Takifugu rubripes': [31033], 'Thalassiosira pseudonana': [35128], 'Giardia lamblia': [5741]}


In [273]:
with open('../data/TOL/Eukaryota_tree_nodes_taxID.txt', 'w') as f:
    for taxa in name2taxid:
        print(taxa, '\t', *name2taxid[taxa], file=f, end='\n')

Search the Entrez Protein database:

In [312]:
def protein_entrez(taxid):
    term = ''.join(['txid', taxid, '[ORGN]'])
    handle = Entrez.esearch(db="protein", term=term)
    record = Entrez.read(handle)
    
    try:
        count = record['Count']
        handle = Entrez.esearch(db="protein", term=term, retmax=count)
        record = Entrez.read(handle)
        return record['IdList']

    except IncompleteRead:
        print('count: ', count)
        return None

In [314]:
for taxa in name2taxid:
    protein_ids = protein_entrez(str(name2taxid[taxa][0]))
    
    if protein_ids:
        path = os.path.join('../data/for_blastdbcmd/GI_proteins/', ''.join([str(name2taxid[taxa][0]), '.gi']))
        with open(path, 'w') as f:
            f.write('\n'.join(protein_ids))

Search for Accession numbers (long run)

In [330]:
i = 1

for taxa in name2taxid:   
    from_path = os.path.join('../data/for_blastdbcmd/GI_proteins/', ''.join([str(name2taxid[taxa][0]), '.gi']))
    to_path = os.path.join('../data/for_blastdbcmd/accessions', ''.join([str(name2taxid[taxa][0]), '.txt']))
    tofile_list = []
    
    with open(from_path, 'r') as fromfile:
        for line in fromfile:
            rec = line.strip()            
            temphandle = Entrez.read(Entrez.esummary(db="protein", id=rec, retmode="text"))
            tofile_list.append(temphandle[0]['Caption'])
    
    with open(to_path, 'w') as tofile:
        tofile.write('\n'.join(tofile_list))
        
    print(i)
    i += 1
    

KeyboardInterrupt: 

In [ ]:
# all to one file
! cd ../data/for_blastdbcmd/accessions
! for file in *.txt; do (cat "${file}"; echo) >> concatenated.txt; done

# Prepare DB

DB https://ftp.ncbi.nih.gov/blast/db/FASTA/   --  nr

In [ ]:
! makeblastdb -in ./blast_db/test_nr.fasta  -dbtype prot -parse_seqids -blastdb_version 5 

Extract part from DB

In [ ]:
# index big fasta DB
! makeblastdb -in nr.fasta -dbtype prot -parse_seqids

# extract
! blastdbcmd -db nr.fasta -dbtype prot -entry_batch concatenated.txt -out ../red_db/reduced_db.fa

# prepare new db
! cd ../red_db/
# delete duplicate in swiss dp
# seqkit rmdup blast_db/red_swiss/swiss_red.fa > blast_db/red_swiss/swiss_red_uniq.fa
! makeblastdb -in reduced_db.fa -dbtype prot -parse_seqids  -blastdb_version 5

# BLAST

https://www.biostars.org/p/208772/

{'Anopheles gambiae': [7165], \
'Arabidopsis thaliana': [3702], \
'Caenorhabditis briggsae': [6238], \
'Caenorhabditis elegans': [6239], \
'Cryptosporidium hominis': [237895], 'Cyanidioschyzon merolae': [45157], 'Danio rerio': [7955], 'Dictyostelium discoideum': [44689], 'Drosophila melanogaster': [7227], 'Eremothecium gossypii': [33169], 'Gallus gallus': [9031], 'Homo sapiens': [9606], 'Leishmania major': [5664], 'Mus musculus': [10090], 'Oryza sativa': [4530], 'Pan troglodytes': [9598], 'Plasmodium falciparum': [5833], 'Rattus norvegicus': [10116], 'Saccharomyces cerevisiae': [4932], 'Schizosaccharomyces pombe': [4896], 'Takifugu rubripes': [31033], 'Thalassiosira pseudonana': [35128], 'Giardia lamblia': [5741]}

In [331]:
# Create taxidlist
with open('../data/TOL/Eukaryota_taxidlist.txids', 'w') as f:
    for taxa in name2taxid:
        print(*name2taxid[taxa], file=f, end='\n')

In [ ]:
! /home/yuliya/soft/ncbi-blast-2.13.0+/bin/blastx -db ./blast_db/test_nr.fasta -query ./genes_lists/S_cerevisiae_genes_seq.fa -taxidlist ./TOL/Eukaryota_taxidlist.txids  -out ./blast/blast_res/blast_result_local.table

In [ ]:
/home/yuliya/soft/ncbi-blast-2.13.0+/bin/blastx -db ./blast_db/swiss/data -query ./genes_lists/S_cerevisiae_genes_seq.fa -num_threads 2 -evalue 1e-3 -out ./blast/blast_res/blast_result_local.table


Local run after reducing DB:
- format output https://www.metagenomics.wiki/tools/blast/blastn-output-format-6

In [ ]:
# run on reduced db (first 1000 seq)
! /home/yuliya/soft/ncbi-blast-2.13.0+/bin/blastx -db blast_db/reduced/reduced.fa -query ./genes_lists/S_cerevisiae_genes_seq.fa -num_threads 2 -evalue 1e-3 -out ./blast/blast_res/blast_eukaryotes_local_3.table -outfmt "6 qseqid sseqid stitle pident evalue bitscore"

In [ ]:
# run on swiss db
! /home/yuliya/soft/ncbi-blast-2.13.0+/bin/blastx -db blast_db/red_swiss/swiss_red_uniq.fa -query ./genes_lists/S_cerevisiae_genes_seq.fa -num_threads 2 -evalue 1e-3 -out ./blast/blast_res/blast_swiss_local.table -outfmt "6 qseqid sseqid stitle pident evalue bitscore"

### remote

In [343]:
new_nodes = []

for node in nodes:
    new_nodes.append(node.strip())

'[ORGN] OR '.join(new_nodes) + '[ORGN]'

'Giardia lamblia[ORGN] OR Leishmania major[ORGN] OR Thalassiosira pseudonana[ORGN] OR Plasmodium falciparum[ORGN] OR Cryptosporidium hominis[ORGN] OR Cyanidioschyzon merolae[ORGN] OR Oryza sativa[ORGN] OR Arabidopsis thaliana[ORGN] OR Dictyostelium discoideum[ORGN] OR Schizosaccharomyces pombe[ORGN] OR Saccharomyces cerevisiae[ORGN] OR Eremothecium gossypii[ORGN] OR Caenorhabditis elegans[ORGN] OR Caenorhabditis briggsae[ORGN] OR Drosophila melanogaster[ORGN] OR Anopheles gambiae[ORGN] OR Danio rerio[ORGN] OR Takifugu rubripes[ORGN] OR Gallus gallus[ORGN] OR Homo sapiens[ORGN] OR Pan troglodytes[ORGN] OR Rattus norvegicus[ORGN] OR Mus musculus[ORGN]'

CMD run:


```/home/yuliya/soft/ncbi-blast-2.13.0+/bin/blastx -db nr -query ./genes_lists/S_cerevisiae_genes_seq.fa -entrez_query "Giardia lamblia[ORGN] OR Leishmania major[ORGN] OR Thalassiosira pseudonana[ORGN] OR Plasmodium falciparum[ORGN] OR Cryptosporidium hominis[ORGN] OR Cyanidioschyzon merolae[ORGN] OR Oryza sativa[ORGN] OR Arabidopsis thaliana[ORGN] OR Dictyostelium discoideum[ORGN] OR Schizosaccharomyces pombe[ORGN] OR Saccharomyces cerevisiae[ORGN] OR Eremothecium gossypii[ORGN] OR Caenorhabditis elegans[ORGN] OR Caenorhabditis briggsae[ORGN] OR Drosophila melanogaster[ORGN] OR Anopheles gambiae[ORGN] OR Danio rerio[ORGN] OR Takifugu rubripes[ORGN] OR Gallus gallus[ORGN] OR Homo sapiens[ORGN] OR Pan troglodytes[ORGN] OR Rattus norvegicus[ORGN] OR Mus musculus [ORGN]"  -out ./blast/blast_res/blast_result.table -remote```



In [342]:
id_list = []
for taxa in name2taxid:
    id_list.append(str(name2taxid[taxa][0]))
    
print(', '.join(id_list))

7165, 3702, 6238, 6239, 237895, 45157, 7955, 44689, 7227, 33169, 9031, 9606, 5664, 10090, 4530, 9598, 5833, 10116, 4932, 4896, 31033, 35128, 5741


# Results analysis

In [84]:
doc = '../data/blast/blast_res/blast_swiss_red_local.table'
result_table = pd.read_csv(doc, sep='\t', header=None, names=['yeast_gene', 'seq_id', 'sub_title', 'p_ident', 'e_value', 'bit_score'])
result_table

,yeast_gene,seq_id,sub_title,p_ident,e_value,bit_score
0,YDL003W,sp|Q12158.1|,RecName: Full=Sister chromatid cohesion protei...,100.000,0.000000e+00,1131.0
1,YDL003W,sp|P30776.1|,RecName: Full=Cohesin subunit rad21; AltName: ...,30.233,9.990000e-11,66.2
2,YDL003W,sp|A2AU37.2|,RecName: Full=Double-strand-break repair prote...,34.783,3.890000e-09,61.2
3,YDL003W,sp|O60216.2|,RecName: Full=Double-strand-break repair prote...,32.967,4.800000e-09,60.8
4,YDL003W,sp|Q61550.3|,RecName: Full=Double-strand-break repair prote...,32.967,5.070000e-09,60.8
...,...,...,...,...,...,...
259993,YDL007W,sp|Q5T9A4.1|,RecName: Full=ATPase family AAA domain-contain...,25.166,3.680000e-05,48.1
259994,YDL007W,sp|F4K7F6.1|,RecName: Full=CLP protease regulatory subunit ...,36.275,6.670000e-05,47.0
259995,YDL007W,sp|Q86XH1.1|,RecName: Full=Dynein regulatory complex protei...,25.444,2.610000e-04,45.4
259996,YDL007W,sp|P38126.2|,RecName: Full=Pachytene checkpoint protein 2 [...,30.539,3.040000e-04,45.1


In [85]:
result_table['species'] = result_table['sub_title'].apply(lambda x: re.findall('(?<!Full\=)\[(.*?)\]', x))
result_table

,yeast_gene,seq_id,sub_title,p_ident,e_value,bit_score,species
0,YDL003W,sp|Q12158.1|,RecName: Full=Sister chromatid cohesion protei...,100.000,0.000000e+00,1131.0,[Saccharomyces cerevisiae S288C]
1,YDL003W,sp|P30776.1|,RecName: Full=Cohesin subunit rad21; AltName: ...,30.233,9.990000e-11,66.2,[Schizosaccharomyces pombe 972h-]
2,YDL003W,sp|A2AU37.2|,RecName: Full=Double-strand-break repair prote...,34.783,3.890000e-09,61.2,[Mus musculus]
3,YDL003W,sp|O60216.2|,RecName: Full=Double-strand-break repair prote...,32.967,4.800000e-09,60.8,[Homo sapiens]
4,YDL003W,sp|Q61550.3|,RecName: Full=Double-strand-break repair prote...,32.967,5.070000e-09,60.8,[Mus musculus]
...,...,...,...,...,...,...,...
259993,YDL007W,sp|Q5T9A4.1|,RecName: Full=ATPase family AAA domain-contain...,25.166,3.680000e-05,48.1,[Homo sapiens]
259994,YDL007W,sp|F4K7F6.1|,RecName: Full=CLP protease regulatory subunit ...,36.275,6.670000e-05,47.0,[Arabidopsis thaliana]
259995,YDL007W,sp|Q86XH1.1|,RecName: Full=Dynein regulatory complex protei...,25.444,2.610000e-04,45.4,[Homo sapiens]
259996,YDL007W,sp|P38126.2|,RecName: Full=Pachytene checkpoint protein 2 [...,30.539,3.040000e-04,45.1,[Saccharomyces cerevisiae S288C]


In [264]:
protein_species = {k: set() for k in result_table['yeast_gene']}

for index, row in result_table.iterrows():
    protein_species[row['yeast_gene']].update(row['species'])

In [265]:
nodes_list=[]

with open('../data/TOL/Eukaryota_tree_nodes.txt', 'r') as file:
    nodes_list = file.readlines()
    for i, node in enumerate(nodes):
        nodes_list[i] = '_'.join(node.strip().split())

In [266]:
for protein in protein_species:
    new_list = []
    for species in protein_species[protein]:
        new_species = species.split()
        if len(new_species) > 1:
            new_species = '_'.join(new_species[:2])
            if new_species in nodes_list:
                new_list.append(new_species)
                
    protein_species[protein] = set(new_list)

In [267]:
# Save results with all ages:
with open('../data/genes_lists/yeast/yeas_protein_species.txt', 'w') as f:
    for protein in protein_species:
        print(protein, '\t', protein_species[protein], file=f, end='\n')

In [268]:
get_linage = dict.fromkeys(nodes_list)

for taxa in name2taxid:
    handle = Entrez.efetch(db='taxonomy', id=name2taxid[taxa], retmode='xml')
    record = Entrez.read(handle, validate=False)
    get_linage['_'.join(taxa.split())] = record[0]['Lineage'].split('; ')
    handle.close()

In [269]:
age_level = dict.fromkeys(get_linage['Saccharomyces_cerevisiae'])

for i, level in enumerate(age_level):
    age_level[level] = i

age_level

{'cellular organisms': 0,
 'Eukaryota': 1,
 'Opisthokonta': 2,
 'Fungi': 3,
 'Dikarya': 4,
 'Ascomycota': 5,
 'saccharomyceta': 6,
 'Saccharomycotina': 7,
 'Saccharomycetes': 8,
 'Saccharomycetales': 9,
 'Saccharomycetaceae': 10,
 'Saccharomyces': 11}

In [270]:
ages_for_node = dict.fromkeys(nodes_list)

for taxa in get_linage:
    for i, level in enumerate(get_linage[taxa]):
        if level == get_linage['Saccharomyces_cerevisiae'][i]:
            mca = level
            continue
        else:
            break
    ages_for_node[taxa] = mca

In [271]:
ages_for_node

{'Giardia_lamblia': 'Eukaryota',
 'Leishmania_major': 'Eukaryota',
 'Thalassiosira_pseudonana': 'Eukaryota',
 'Plasmodium_falciparum': 'Eukaryota',
 'Cryptosporidium_hominis': 'Eukaryota',
 'Cyanidioschyzon_merolae': 'Eukaryota',
 'Oryza_sativa': 'Eukaryota',
 'Arabidopsis_thaliana': 'Eukaryota',
 'Dictyostelium_discoideum': 'Eukaryota',
 'Schizosaccharomyces_pombe': 'Ascomycota',
 'Saccharomyces_cerevisiae': 'Saccharomyces',
 'Eremothecium_gossypii': 'Saccharomycetaceae',
 'Caenorhabditis_elegans': 'Opisthokonta',
 'Caenorhabditis_briggsae': 'Opisthokonta',
 'Drosophila_melanogaster': 'Opisthokonta',
 'Anopheles_gambiae': 'Opisthokonta',
 'Danio_rerio': 'Opisthokonta',
 'Takifugu_rubripes': 'Opisthokonta',
 'Gallus_gallus': 'Opisthokonta',
 'Homo_sapiens': 'Opisthokonta',
 'Pan_troglodytes': 'Opisthokonta',
 'Rattus_norvegicus': 'Opisthokonta',
 'Mus_musculus': 'Opisthokonta'}

In [272]:
yest_gene_ages_lists = {k: [] for k in result_table['yeast_gene']}
yest_gene_ages = {k: [] for k in result_table['yeast_gene']}

for protein in protein_species:
    temp = []
    for species in protein_species[protein]:
        yest_gene_ages_lists[protein].append(ages_for_node[species])
        temp.append(age_level[ages_for_node[species]])
        
    yest_gene_ages[protein] = get_linage['Saccharomyces_cerevisiae'][min(temp)]

In [281]:
yeast_age_hits = pd.DataFrame(columns=['Gene_ID', *get_linage['Saccharomyces_cerevisiae'][1:]])

for i, gene in enumerate(yest_gene_ages_lists):
    yeast_age_hits.loc[gene] = [gene, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    for level in yest_gene_ages_lists[gene]:
        yeast_age_hits.loc[gene, level] += 1

In [282]:
yeast_age_hits

,Gene_ID,Eukaryota,Opisthokonta,Fungi,Dikarya,Ascomycota,saccharomyceta,Saccharomycotina,Saccharomycetes,Saccharomycetales,Saccharomycetaceae,Saccharomyces
YDL003W,YDL003W,1,3,0,0,1,0,0,0,0,0,1
YKR060W,YKR060W,0,2,0,0,1,0,0,0,0,0,1
YMR254C,YMR254C,0,0,0,0,0,0,0,0,0,0,1
YEL059W,YEL059W,0,0,0,0,0,0,0,0,0,0,1
YHR030C,YHR030C,4,8,0,0,1,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
YKL037W,YKL037W,0,0,0,0,0,0,0,0,0,0,1
YOR209C,YOR209C,1,1,0,0,1,0,0,0,0,0,1
YHR005C,YHR005C,3,7,0,0,1,0,0,0,0,0,1
YPR189W,YPR189W,1,2,0,0,1,0,0,0,0,1,1


In [297]:
# Save results with all ages:
with open('../data/genes_lists/yeast/yest_gene_ages_lists.txt', 'w') as f:
    for gene in yest_gene_ages_lists:
        print(gene, *yest_gene_ages_lists[gene], file=f, end='\n')
        
# Save results with all ages:
with open('../data/genes_lists/yeast/yest_gene_ages.txt', 'w') as f:
    for gene in yest_gene_ages:
        print(f'{gene}\t{yest_gene_ages[gene]}', file=f, end='\n')
        
        
yeast_age_hits.to_csv('../data/genes_lists/yeast/yeast_age_hits.csv', index=False)

In [298]:
get_linage['Saccharomyces_cerevisiae']

['cellular organisms',
 'Eukaryota',
 'Opisthokonta',
 'Fungi',
 'Dikarya',
 'Ascomycota',
 'saccharomyceta',
 'Saccharomycotina',
 'Saccharomycetes',
 'Saccharomycetales',
 'Saccharomycetaceae',
 'Saccharomyces']